In [1]:
from tensorflow.keras import models, layers, datasets, applications
from sklearn.preprocessing import OneHotEncoder
import numpy as np

In [2]:
input_tensor = layers.Input(shape=(32, 32, 3))

model = applications.ResNet50(input_shape=(32, 32, 3), include_top=False, weights='imagenet',pooling=None, classes=10)
model.summary()

94773248/94765736 [==============================] - 58s 1us/step
Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 38, 38, 3)    0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 16, 16, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 16, 16, 64)   256         conv1_conv[0][0]                 
_________________________

In [3]:
x = model.output
x

<tf.Tensor 'conv5_block3_out/Identity:0' shape=(None, 1, 1, 2048) dtype=float32>

In [11]:
# 加入一層
x = layers.Dense(units=128, activation='relu')(x)
x = layers.Dropout(0.1)(x)
outputs = layers.Dense(units=10, activation='softmax')(x)

model = models.Model(inputs=model.input, outputs=outputs)
print('深度: {}'.format(len(model.layers)))

深度: 182


In [12]:
# 鎖定特定幾層不要更新權重
# 前一百不要更新
for layer in model.layers[:100]:
    layer.trainable = False
for layer in model.layers[100:]:
    layer.trainable = True

In [13]:
(x_train, y_train), (x_test, y_test) = datasets.cifar10.load_data()

def normalize(X_train,X_test):
    mean = np.mean(X_train,axis=(0,1,2,3))
    std = np.std(X_train, axis=(0, 1, 2, 3))
    X_train = (X_train-mean)/(std+1e-7)
    X_test = (X_test-mean)/(std+1e-7)
    return X_train, X_test

# 編碼OneHot Label 由(None, 1)-(None, 10)
x_train, x_test = normalize(x_train, x_test) 
one_hot = OneHotEncoder()
y_train = one_hot.fit_transform(y_train).toarray()
y_test = one_hot.transform(y_test).toarray()

In [14]:
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(50000, 32, 32, 3) (50000, 10)
(10000, 32, 32, 3) (10000, 10)


In [15]:
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [17]:
# 訓練，測試使用
model.fit(x_train, y_train, batch_size=32, epochs=1)

1563/1563 [==============================] - 97s 62ms/step - loss: 1.5079 - accuracy: 0.4691
